# Interactive tutorial on using the functions witin the optical_mutlilayer (om) module.
### Created by Dr. William Robertson and David C. Heson. 

<p>Initial version of the tutorial, more explanations and examples will be added + any new features will receive their own section.<br>

Please email any suggestions or questions to [Dr. William Robertson](mailto:william.robertson@mtsu.edu) or [David C. Heson](mailto:dch376@msstate.edu).</p>

### Code features as listed in this tutorial:
<ol>
    <li>Taking as input material multilayers of any custom lenghts, number of layers, and with any custom indeces of refraction (which can be complex).</li>
    <li>Simulating intensity of reflected light through a multilayer with respect to the incident angle of the inbound light at a set wavelength.</li>
    <li>Simulating intensity of reflected light through a multilayer with respect to the wavelength of the inbound light at a set incident angle.</li>
    <li>Simulating intensity of reflected light through a multilayer with respect to the wavelength of the inbound light at a set incident angle, while using a known relationship between wavelength and index of refraction to adjust the indeces dynamically. (advanced)</li>
    <li>Overview of intensity of reflected or transmitted light through a multilayer simulation options.</li>
    <li>Simulating the electric field through a multilayer at a fixed wavelength and incident angle for the inbound light.</li>
    <li>Overview of core multilayer simulation functionalities.</li>
    <li>Practical examples of light intensity and electric field simulations.</li>
    <li>Using the inbuilt graphing methods for electric field simulations.</li>
    <li>Using the inbuilt Bloch Surface Wave detector. (unstable)</li>
    <li>Using the inbuilt Refractive Index Unit (RIU) / Shift With Thickness (SWT) calculators. (unstable)
    <li>Using the inbuilt brute-force multilayer designer. (advanced)(unstable)</li>
</ol>

<p>Let's get started.<br>
Below we will just import the Python modules we need.</p>

In [ ]:
# importing the modules which will be used in this tutorial
# the optical_multilayer module itself operates off numpy and matplotlib, 
# but importing these is useful for easily visualizing the generated simulations
# and send input arguments which are more stable for the code to "digest"

import numpy as np
import optical_multilayer as om
import matplotlib.pyplot as plt

# for this to work make sure you have installed these python packages.
# this can be easily done by using the pip command in your terminal
# i.e. "pip install optical_multilayer"
# for additional help you can visit the Python Package Index (PIP) website

The main utility of the program is the simulation of the light intensity and electric field profile accross any user defined multilayers. The code is extremely flexible, and can handle a large array of parameters. This is also arguably its biggest downside, making it slightly convoluted at first.
For this reason, we will attempt to break down each component of the provided functions step by step.

The conceptual difficulty of the physics involved only futher convolutes the process. If you are not familiar with the topics at hand and wish to get an understanding of the physics involved, 

### 1. Taking as input material multilayers of any custom lenghts, number of layers, and with any custom indeces of refraction (which can be complex

To begin with, we will simply look at how to effectively make the variables which will be used to define the various simulation parameters. 
The main multilayer function has a lot of parameters, for now we will simply focus on the basic ones which are required for virtually all simulations.

In [ ]:
# The main function with its default arguments is as following:
# om.multilayer(n_list, dist, mode = 'rp', step = 1000, option_aw = 'a', e_opt = 'n',
#               a1 = 0, a2 = np.pi/2, w_t = 650e-9, w1 = 400e-9, w2 = 750e-9,
#               ang = np.pi/3, n_wv_dic = {}, nw_list = [], limit = 1000e-9)

# n_list represents the list of refractive indeces for the mediums considered, in order
# from top to bottom. parsing this number as conmplex even if the complex part is 0 is
# strongly reccomended

# dist is the thicknesses of the multilayers involved, and they need to be two less than 
# the number of mediums specifies with n_list, since the two outside mediums are considered
# infinite

# mode represents the polarisation mode and whether transmission or reflection coefficients 
# are calculated (the difference between them giving the absorption coefficients)
# the first letter stands for (r)eflection or (t)ransmission, and the second letter stands
# for either P(parallel) or S(perpendicular) polarisation

# this is a lot of words, so to visualize it, consider a simple system that has 
# glass on top (n = 1.5, 0.00), Silicon Dioxide (Si02) with n = 1.457, 3.4e-5
# and then a Titanium Dioxie (TiO2) with n = 2.3, 1.5e-4, SiO2 again, TiO2 again, 
# and then with the bottom layer being water (n = 1.33, 0.00)
# Let the thickness of the first layer be 132 nm, the thickness of the 2nd layer be 250 nm, 
# thickness of the 3rd layer be 132 nm again, and the thickness of the final layer be 
# 300 nm. We will calculate the reflection coefficient with P polarized light

# we will use a resolution of 10000 steps for this, which means the simulation will 
# generate a total of 10000 points at which it will evaluate the corresponding coefficients

# the set-up code for this scenario would look like:

index = np.array([complex(1.5, 0.00), complex(1.457, 3.4e-5), complex(2.3, 1.5e-4),
        complex(1.457, 3.4e-5), complex(2.3, 1.5e-4), complex(1.33, 0.00)])
        ### variable defining the indeces of refraction in the system
    
depth = np.array([132e-9, 250e-9, 132e-9, 300e-9])
        ### variable defining the depths of the multilayers in the system

res = 10000
        ### variable giving the "resolution" (or number of simulated points)
    
pol = 'rs'
        ### variable giving whether the simulation shows transmission or reflection
        ### coefficients and the polarisation mode of the light
        ### future versions will support light with mixed polarisation states
        ### the polarisation mode key is:
            ### rs - S-Polarized Inbound to Reflected Light Intensity 
            ### rp - P-Polarized Inbound to Reflected Light Intensity
            ### ts - S-Polarized Inbound to Transmitted Light Intensity 
            ### tp - P-Polarized Inbound to Transmitted Light Intensity
        ### note that this variable *has* to be one of these specific strings
        ### otherwise the code will return an exception
        
# note that the layer variables are defined with np.array, numpy arrays being more efficient
# than classic python lists, and numpy has better complex number support hence avoids some
# weird bugs from arising
# side note: to easily express exponents in python, you can use the "e" symbol in the 
# number, as it can be seen in the definition of the layer depth 

# side note: the thicknesses and refraction coefficients used do not necessarily 
# corellate to any real materials or systems, they were random choices

### 2. Simulating intensity of reflected light through a multilayer with respect to the incident angle of the inbound light at a set wavelength.

We will simply utilize the hyperparameters defined in the section above to do this. Consider a set wavelength of inbound light of 450 nm for this simulation. Our output will be a graph showing the intensity of light reflected out of the system as a function of changing incident angle. The code is as follows:

In [ ]:
wave = 450e-9
    ### variable defining the incident wavelength of light
    
a1 = 0
    ### angle of incident light at which the simulation starts
a2 = np.pi/2 
    ### angle of incident light at which the simulation ends
    ### note that this has to be in radians
    
output = om.multilayer(index, depth, mode = pol, step = res, option_aw = 'a',
               a1 = a1, a2 = a2, w_t = wave)
    ### the simulation should run fairly quickly with a res of only 10000 steps
    ### however, this time can increase significantly depending on your machine
    ### if a large number of steps is chosen
    ### additionaly, note that the w_t variable gives the constant wavelength at which we
    ### wish the simulation to run
    ### additionally, note the "option_aw" parameter. this one determines whether or not
    ### the simulation runs with respect to changing angle or changing wavelength
    ### the default is set to changing angle, but it is always good to specify it

In [ ]:
# let us now investigate what the function *actually* returns
output

In [ ]:
# as you can see, the returned output is a numpy array itself
# it is hard to really understand its meaning by just looking at it,
# but we can see that it has 2 columns, which means we can make a
# 2D graph of the data it represents. let's use matplotlib to do so

plt.plot(output[:,0], output[:,1])

In [ ]:
# the output is a graph that shows the intensity of the reflected light, relative to the 
# inclination of the inbound light
# we can see from this that past a specific angle, we see total internal reflection

# now, let's make a nicer graph, with a x-axis in degrees

angles = output[:,0] * (180/np.pi)
intensity = output[:,1]

fig, ax = plt.subplots(figsize=(12, 6))
plt.plot(angles, intensity, label = "Inbound light at 450 nm")
plt.title("Ratio of light reflected from S-polarized light")
plt.xlabel("Inbound Light Angle (Degrees)")
plt.ylabel("Intensity of Reflected Light Relative to Inbound Light")
plt.legend()
plt.grid()
plt.show()

We have now succesfully simulated the amount of light reflected out of an optical multilayer with respect to diferent inbound angles!

### 3. Simulating intensity of reflected light through a multilayer with respect to the wavelength of the inbound light at a set incident angle.

This process will be extremely similar to the fixed wavelength one. The same hyperparameters (layer depths, medium indeces, light polarisation and reflection) will be used from now on unless otherwise specified.

In [ ]:
w1 = 300e-9
    ### starting wavelength of simulation at 300 nm
w2 = 700e-9
    ### ending wavelength of simulation at 700 nm
ang = 62.5 * np.pi/180
    ### fixed angle of simulation, note that we must make sure that it is in radians

output = om.multilayer(index, depth, mode = pol, step = res, option_aw = 'w',
               w1 = w1, w2 = w2, ang = ang)

fig, ax = plt.subplots(figsize=(12, 6))
plt.plot(output[:,0] * 10e8, output[:,1], label = "Inbound light at 62.5 degrees")
plt.title("Ratio of light reflected from S-polarized light")
plt.xlabel("Inbound Light Wavelength (nm)")
plt.ylabel("Intensity of Reflected Light Relative to Inbound Light")
plt.legend()
plt.grid()
plt.show()

### 4. Simulating intensity of reflected light through a multilayer with respect to the wavelength of the inbound light at a set incident angle, while using a known relationship between wavelength and index of refraction to adjust the indeces dynamically. (advanced)

In reality, indeces of refraction are not constant. There is an inbuilt functionality in the code which automatically adjusts the indeces of refraction of the mediums, given that each index of refraction can be expressed in the form of a formula which depends solely on the wavelength of incident light.

We will use Python dictionaries for this, and have to define our own index of refraction functions.

In [ ]:
# we will use the same wavelength range and fixed angle as in the section before

# to start easy, we will first define the index functions for glass and water, because
# their indeces will simply be constant

def glass(lmbd):
    return complex(1.5,0)
def water(lmbd):
    return complex(1.33,0)
# the function needs to take a wavelength in meters as input, and return the 
# index of refraction as a complex number

# now for the more complex equations for materials Si02 and TiO2, these will be:

def tio2(lmbd):
    lmbd0 = lmbd * (10 ** (6))
    out = complex(np.sqrt(5.913 + (0.2441) / (-0.0803 + lmbd0 ** 2)), 0.0007)
    return out
           
def sio2(lmbd):
    lmbd0 = lmbd * (10 ** (6))
    a = (0.6961663 * (lmbd0 ** 2)) / ((lmbd0 ** 2) - (0.0684043 ** 2))
    b = (0.4079462 * (lmbd0 ** 2)) / ((lmbd0 ** 2) - (0.1162414 ** 2))
    c = (0.8974794 * (lmbd0 ** 2)) / ((lmbd0 ** 2) - (9.896161 ** 2))
    out = complex(np.sqrt(a + b + c + 1), 0.0001)
    return out

# a lot of index of refraction equations use wavelength in terms of microns, but the 
# input has to be in meters, so if that is the case we have to convert in the functions
# the wavelength from meters to microns

# now for the more complicated aspect of this, we will create a dispatch list and
# the function dictionary

# the function dictionary will simply assign a string value to each function, so that
# it can be easily parsed through the main function
func_dic = {'glass': glass, 'tio2': tio2, 'sio2': sio2, 'water': water}

# the dispatch list will indicate the order in which the main function needs to call
# index of refraction functions from the function dictionary
dispatch_list = ['glass', 'tio2', 'sio2', 'tio2', 'sio2', 'water']

# now to actually run the simulation...

output = om.multilayer(index, depth, mode = pol, step = res, option_aw = 'w',
               w1 = w1, w2 = w2, ang = ang, n_wv_dic = func_dic, nw_list = dispatch_list)
        ### n_wv_dic is the dictionary variable in the main function. it is empty by
        ### default, and if the function is parsed through with it empty, the code knows 
        ### we are not using dynamic indeces. nw_list is the function dispatch list

fig, ax = plt.subplots(figsize=(12, 6))
plt.plot(output[:,0] * 10e8, output[:,1], label = "Inbound light at 62.5 degrees")
plt.title("Ratio of light reflected from S-polarized light")
plt.xlabel("Inbound Light Wavelength (nm)")
plt.ylabel("Intensity of Reflected Light Relative to Inbound Light")
plt.legend()
plt.grid()
plt.show()

### 8. Practical examples of light intensity and electric field simulations.

In [ ]:
# Simulation of the reflectivity versus angle of a 50 nm silver film
# in a prism/silver/air configuration

index = np.array([complex(1.5,0.0), complex(0.03,4.24),complex(1.0,0.0)])
thickness = np.array([complex(50e-9,0.0)])

angstart = 40*(np.pi/180)
angend = 50*(np.pi/180)

R = om.multilayer(index, thickness, mode = pol, step = 5000, option_aw = 'a', e_opt = 'n',
               a1 = angstart, a2 = angend, w_t = 650e-9, w1 = 400e-9, w2 = 750e-9,
               ang = np.pi/3, n_wv_dic = {}, nw_list = [], limit = 1000e-9)

plt.plot(R[:,0]*57.2598,R[:,1])
plt.xlabel('Incident Angle (degrees)')
plt.ylabel('Reflectivity')
plt.title('Reflectivity versus angle for 50 nm silver film')
plt.grid()
plt.show()

In [ ]:
# Simulation of the reflectivity versus wavelength of a 50 nm silver film
# in a prism/silver/air configuration


angle = 43.35*(np.pi/180)
W = om.multilayer(index, thickness, mode = 'rp', step = 5000, option_aw = 'w', e_opt = 'n',
               a1 = np.pi/6, a2 = np.pi/2, w_t = 650e-9, w1 = 300e-9, w2 = 850e-9,
               ang = angle, n_wv_dic = {}, nw_list = [], limit = 1000e-9)

plt.plot(W[:,0]*1e9,W[:,1])
plt.xlabel('Incident wavelength (nm)')
plt.ylabel('Reflectivity')
plt.title('Reflectivity versus wavelength for 50 nm silver film')
plt.grid()
plt.show()

In [ ]:
# Simulation of cross-sectional electromagnetic field intensity (e_opt = 'y') for 50 nm
# silver film. The calculation is done at 650 nm at the resonant angle for surface
# plasmon generation as determined in the first plot.

Ef = om.multilayer(index, thickness, mode = 'rp', step = 5000, option_aw = 'w', e_opt = 'y',
               a1 = np.pi/6, a2 = np.pi/2, w_t = 650e-9, w1 = 300e-9, w2 = 850e-9,
               ang = angle, n_wv_dic = {}, nw_list = [], limit = 1000e-9)

plt.plot(Ef[:,0]*1e9,Ef[:,1])
plt.xlabel('Cross-sectional distance (nm)')
plt.ylabel('Field intensity (|E|^2)')
plt.title('Cross-sectional field intensity at resonance for 50 nm silver film')
plt.grid()
plt.show()

In [ ]:
# Bloch surface wave simulation for a SiO2/TiO2 multilayer.

ntio = np.sqrt(4.84+0.0007*1j)
nsio = np.sqrt(2.1316+0.0001*1j)
dtio = complex(121e-9,0)
dsio = complex(188e-9,0)
ddefect= complex(410e-9,0)

indexB = np.array([complex(1.5,0),ntio,nsio,ntio,nsio,ntio,nsio,ntio,nsio,complex(1.0,0)])
thicknessB = np.array([dtio,dsio,dtio,dsio,dtio,dsio,dtio,ddefect])
angstart=61*(np.pi/180)
angend=63*(np.pi/180)

RBSW = om.multilayer(indexB, thicknessB, mode = 'rs', step = 10000, option_aw = 'a', e_opt = 'n',
               a1 = angstart, a2 = angend, w_t = 650e-9, w1 = 400e-9, w2 = 750e-9,
               ang = np.pi/3, n_wv_dic = {}, nw_list = [], limit = 1000e-9)

plt.plot(RBSW[:,0]*57.2598,RBSW[:,1])
plt.xlabel('Angle (degrees)')
plt.ylabel('Reflectivity')
plt.title('Angular reflectivity of prism-coupled Bloch surf. wave')
plt.grid()
plt.show()

In [ ]:
# BSW Field profile. The cross-sectional EM filed intensity is plotted at the resonant angle
# found in the previous plot. Note that the field is normalized to E=1 in the initial medium.

angleB = 62.25*(np.pi/180)

EfBSW = om.multilayer(indexB, thicknessB, mode = 'rs', step = 10000, option_aw = 'w', 
                      e_opt = 'y', a1 = np.pi/6, a2 = np.pi/2, w_t = 650e-9, 
                      w1 = 300e-9, w2 = 850e-9, ang = angleB, n_wv_dic = {}, nw_list = [], 
                      limit = 1000e-9)

Ind = np.round(np.sum(thicknessB)*1e9)
xplot = np.array([Ind,Ind+1])
ymax=np.max(EfBSW[:,1])
yplot=np.array([0,ymax])

plt.plot(EfBSW[:,0]*1e9,EfBSW[:,1])
plt.plot(xplot,yplot)
plt.xlabel('Cross-sectional distance (nm)')
plt.ylabel('Field intensity (|E|^2)')
plt.title('Cross-sectional field intensity at resonance for BSW multilayer')
plt.grid()
plt.show()

[1]: <https://pypi.org/project/optical-multilayer/> "Python Package Index Page"